In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shanj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shanj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shanj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Ranking applicants according to the employers requirement

In [7]:

dataset_df = pd.read_csv(r'C:\Users\shanj\Downloads\potentialtalents.csv')

# Expanding abbreviations

In [8]:
abbreviation_dict = {
    "HR": "Human Resource",
    
}

# Function to expand abbreviations in a sentence
def expand_abbreviations(sentence):
    words = sentence.split()
    expanded_sentence = [abbreviation_dict.get(word, word) for word in words]
    return ' '.join(expanded_sentence)

# Apply the function to the entire 'sentences' column
dataset_df['expanded_sentences'] = dataset_df['job_title'].apply(expand_abbreviations)

# Display the original and expanded sentences
print(dataset_df[['job_title', 'expanded_sentences']])

                                             job_title  \
0    2019 C.T. Bauer College of Business Graduate (...   
1    Native English Teacher at EPIK (English Progra...   
2                Aspiring Human Resources Professional   
3               People Development Coordinator at Ryan   
4      Advisory Board Member at Celal Bayar University   
..                                                 ...   
99   Aspiring Human Resources Manager | Graduating ...   
100              Human Resources Generalist at Loparex   
101   Business Intelligence and Analytics at Travelers   
102                     Always set them up for Success   
103   Director Of Administration at Excellence Logging   

                                    expanded_sentences  
0    2019 C.T. Bauer College of Business Graduate (...  
1    Native English Teacher at EPIK (English Progra...  
2                Aspiring Human Resources Professional  
3               People Development Coordinator at Ryan  
4      Advisory Bo

In [9]:
dataset_df.head(25)

,id,job_title,location,connection,fit,expanded_sentences
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,2019 C.T. Bauer College of Business Graduate (...
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN,Native English Teacher at EPIK (English Progra...
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,People Development Coordinator at Ryan
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,Advisory Board Member at Celal Bayar University
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist
6,7,Student at Humber College and Aspiring Human R...,Kanada,61,NaN,Student at Humber College and Aspiring Human R...
7,8,HR Senior Specialist,San Francisco Bay Area,500+,NaN,Human Resource Senior Specialist
8,9,Student at Humber College and Aspiring Human R...,Kanada,61,NaN,Student at Humber College and Aspiring Human R...
9,10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN,Seeking Human Resources HRIS and Generalist Po...


In [12]:
target_sentences = ["Aspiring human resources"]


# Processing the column by tokenizing and lematizing

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')

# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(sentence):
    tokens = word_tokenize(sentence)
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Modify the cosine similarity calculation function
def calculate_cosine_similarity(corpus, target_sentence):
    corpus = [tokenize_and_lemmatize(sentence) for sentence in corpus]
    target_sentence = tokenize_and_lemmatize(target_sentence)

    vectorizer = CountVectorizer().fit_transform([target_sentence] + corpus)
    vectors = vectorizer.toarray()
    similarity_scores = cosine_similarity(vectors)
    return similarity_scores[1:]

# Tokenize and lemmatize the 'expanded_sentences' column
dataset_df['processed_expanded_sentences'] = dataset_df['expanded_sentences'].apply(tokenize_and_lemmatize)

# Display the DataFrame with the processed column
print("Processed DataFrame:")
print(dataset_df[['processed_expanded_sentences']])




Processed DataFrame:
                          processed_expanded_sentences
0    2019 c.t . bauer college of business graduate ...
1    native english teacher at epik ( english progr...
2                 aspiring human resource professional
3               people development coordinator at ryan
4      advisory board member at celal bayar university
..                                                 ...
99   aspiring human resource manager | graduating m...
100               human resource generalist at loparex
101    business intelligence and analytics at traveler
102                     always set them up for success
103   director of administration at excellence logging

[104 rows x 1 columns]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shanj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shanj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
dataset_df

,id,job_title,location,connection,fit,expanded_sentences,processed_expanded_sentences,cosine_similarity_scores
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,2019 C.T. Bauer College of Business Graduate (...,2019 c.t . bauer college of business graduate ...,0.462910
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN,Native English Teacher at EPIK (English Progra...,native english teacher at epik ( english progr...,0.000000
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,People Development Coordinator at Ryan,people development coordinator at ryan,0.000000
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,Advisory Board Member at Celal Bayar University,advisory board member at celal bayar university,0.000000
...,...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN,Aspiring Human Resources Manager | Graduating ...,aspiring human resource manager | graduating m...,0.629941
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN,Human Resources Generalist at Loparex,human resource generalist at loparex,0.516398
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,Business Intelligence and Analytics at Travelers,business intelligence and analytics at traveler,0.000000
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN,Always set them up for Success,always set them up for success,0.000000


In [20]:
# Calculate and print cosine similarity
dataset_df['cosine_similarity_scores_Aspiring'] = dataset_df['processed_expanded_sentences'].apply(lambda x: calculate_cosine_similarity([x], target_sentences[0])[0][0])

print("\nCosine Similarity Scores Aspiring:")
print(dataset_df[['cosine_similarity_scores_Aspiring']])


Cosine Similarity Scores Aspiring:
     cosine_similarity_scores_Aspiring
0                             0.462910
1                             0.000000
2                             0.866025
3                             0.000000
4                             0.000000
..                                 ...
99                            0.629941
100                           0.516398
101                           0.000000
102                           0.000000
103                           0.000000

[104 rows x 1 columns]


In [24]:
target_sentence = ["Seeking human resources"]

dataset_df['cosine_similarity_scores_seeking'] = dataset_df['processed_expanded_sentences'].apply(lambda x: calculate_cosine_similarity([x], target_sentence[0])[0][0])

print("\nCosine Similarity Scores seeking:")
print(dataset_df[['cosine_similarity_scores_seeking']])


Cosine Similarity Scores seeking:
     cosine_similarity_scores_seeking
0                            0.308607
1                            0.000000
2                            0.577350
3                            0.000000
4                            0.000000
..                                ...
99                           0.629941
100                          0.516398
101                          0.000000
102                          0.000000
103                          0.000000

[104 rows x 1 columns]


In [26]:
dataset_df['max_values'] = dataset_df[['cosine_similarity_scores_seeking', 'cosine_similarity_scores_Aspiring']].max(axis=1)



In [28]:
pd.set_option('display.max_rows', 500)
dataset_df.sort_values(by='max_values',ascending=False)

,id,job_title,location,connection,fit,expanded_sentences,processed_expanded_sentences,cosine_similarity_scores_Aspiring,cosine_similarity_scores_seeking,max_values
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,Seeking Human Resources Opportunities,seeking human resource opportunity,0.577350,0.866025,0.866025
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,NaN,Seeking Human Resources Position,seeking human resource position,0.577350,0.866025,0.866025
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,Seeking Human Resources Opportunities,seeking human resource opportunity,0.577350,0.866025,0.866025
96,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025


# Ranking the applicants according to similarity to employers requirement

In [29]:
dataset_df.sort_values(by='max_values',ascending=False)

,id,job_title,location,connection,fit,expanded_sentences,processed_expanded_sentences,cosine_similarity_scores_Aspiring,cosine_similarity_scores_seeking,max_values
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,Seeking Human Resources Opportunities,seeking human resource opportunity,0.577350,0.866025,0.866025
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,NaN,Seeking Human Resources Position,seeking human resource position,0.577350,0.866025,0.866025
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,Seeking Human Resources Opportunities,seeking human resource opportunity,0.577350,0.866025,0.866025
96,97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025


In [31]:
tgs = ["Aspiring Human Resources Specialist"]
dataset_df['cosine_similarity_scores_AHRS'] = dataset_df['processed_expanded_sentences'].apply(lambda x: calculate_cosine_similarity([x], tgs[0])[0][0])

print("\nCosine Similarity Scores AHRS:")
print(dataset_df[['cosine_similarity_scores_AHRS']])


Cosine Similarity Scores AHRS:
     cosine_similarity_scores_AHRS
0                         0.400892
1                         0.000000
2                         0.750000
3                         0.000000
4                         0.000000
5                         1.000000
6                         0.500000
7                         0.750000
8                         0.500000
9                         0.377964
10                        0.000000
11                        0.000000
12                        0.377964
13                        0.400892
14                        0.400892
15                        0.000000
16                        0.750000
17                        0.000000
18                        0.400892
19                        0.000000
20                        0.750000
21                        0.000000
22                        0.000000
23                        1.000000
24                        0.500000
25                        0.750000
26                     

# Ranking applicants according to their similarity to a different requirement

In [32]:
dataset_df.sort_values(by='cosine_similarity_scores_AHRS',ascending=False)

,id,job_title,location,connection,fit,expanded_sentences,processed_expanded_sentences,cosine_similarity_scores_Aspiring,cosine_similarity_scores_seeking,max_values,cosine_similarity_scores_AHRS
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025,1.000000
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025,1.000000
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025,1.000000
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025,1.000000
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,Aspiring Human Resources Specialist,aspiring human resource specialist,0.866025,0.577350,0.866025,1.000000
60,61,HR Senior Specialist,San Francisco Bay Area,500+,NaN,Human Resource Senior Specialist,human resource senior specialist,0.577350,0.577350,0.577350,0.750000
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025,0.750000
20,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025,0.750000
50,51,HR Senior Specialist,San Francisco Bay Area,500+,NaN,Human Resource Senior Specialist,human resource senior specialist,0.577350,0.577350,0.577350,0.750000
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,Aspiring Human Resources Professional,aspiring human resource professional,0.866025,0.577350,0.866025,0.750000


# Conclusion

Applicants where ranked comparing the similarity between their bio and the requirement proposed by the employer.

First requriement: Aspiring human resources
Second: Seeking human resource

Bio where processed by performing tokenization, expansion and lematization and the cosine similarity for each bio was calculated.

Applicants where further ranked according to the cosine similarity.

Applicant number 27 was found to be the best applicant.

Further another reauirement was considered and ranked the applicants accordingly.
